In [11]:
import psycopg2
import csv

def create_table():
    conn = psycopg2.connect(dbname='PhoneBook', user='postgres', password='1234', host='localhost', port='5432')
    cur = conn.cursor()
    cur.execute('''
        CREATE TABLE IF NOT EXISTS phonebook(
            id SERIAL PRIMARY KEY,
            name VARCHAR(100),
            phone VARCHAR(20)
        );
    ''')
    conn.commit()
    cur.close()
    conn.close()
    print('Table is created')

def insert_from_console():
    name = input("Your name: ")
    phone = input("Your phone number: ")
    conn = psycopg2.connect(dbname='PhoneBook', user='postgres', password='1234', host='localhost', port='5432')
    cur = conn.cursor()
    cur.execute("SELECT * FROM phonebook WHERE name = %s", (name,))
    if cur.fetchone():
        cur.execute("UPDATE phonebook SET phone = %s WHERE name = %s", (phone, name))
        print("Phone updated")
    else:
        cur.execute("INSERT INTO phonebook (name, phone) VALUES (%s, %s)", (name, phone))
        print("New user added")
    conn.commit()
    cur.close()
    conn.close()

def insert_from_csv(filename):
    try:
        with open(filename, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            conn = psycopg2.connect(dbname='PhoneBook', user='postgres', password='1234', host='localhost', port='5432')
            cur = conn.cursor()
            invalid_data = []
            for row in reader:
                if len(row) >= 2:
                    name, phone = row[0], row[1]
                    if phone.isdigit() and 6 <= len(phone) <= 15:
                        cur.execute("SELECT * FROM phonebook WHERE name = %s", (name,))
                        if cur.fetchone():
                            cur.execute("UPDATE phonebook SET phone = %s WHERE name = %s", (phone, name))
                        else:
                            cur.execute("INSERT INTO phonebook (name, phone) VALUES (%s, %s)", (name, phone))
                    else:
                        invalid_data.append(f"{name}:{phone}")
            conn.commit()
            cur.close()
            conn.close()
            print(f"Data imported from {filename}")
            if invalid_data:
                print("Invalid entries:", invalid_data)
    except FileNotFoundError:
        print("File is not found")
    except Exception as e:
        print(f"Error: {e}")

def query_data():
    print("1. Search by pattern")
    print("2. Show all records")
    print("3. Show paginated records")
    choice = input("Choose option (1-3): ")
    conn = psycopg2.connect(dbname='PhoneBook', user='postgres', password='1234', host='localhost', port='5432')
    cur = conn.cursor()

    if choice == '1':
        pattern = input("Enter search pattern: ")
        cur.execute("SELECT * FROM phonebook WHERE name ILIKE %s OR phone ILIKE %s", (f"%{pattern}%", f"%{pattern}%"))
    elif choice == '2':
        cur.execute("SELECT * FROM phonebook")
    elif choice == '3':
        limit = int(input("Enter limit: "))
        offset = int(input("Enter offset: "))
        cur.execute("SELECT * FROM phonebook ORDER BY id LIMIT %s OFFSET %s", (limit, offset))
    else:
        print("Invalid choice")
        return

    records = cur.fetchall()
    if not records:
        print("No records found")
    else:
        for row in records:
            print(f"ID: {row[0]}, Name: {row[1]}, Phone: {row[2]}")
    cur.close()
    conn.close()

def update_data():
    print("1. Change name")
    print("2. Change phone")
    choice = input("Choose option (1/2): ")
    conn = psycopg2.connect(dbname='PhoneBook', user='postgres', password='1234', host='localhost', port='5432')
    cur = conn.cursor()

    if choice == '1':
        old_name = input("Enter current name: ")
        new_name = input("Enter new name: ")
        cur.execute("UPDATE phonebook SET name = %s WHERE name = %s", (new_name, old_name))
    elif choice == '2':
        name = input("Enter name: ")
        new_phone = input("Enter new phone number: ")
        cur.execute("UPDATE phonebook SET phone = %s WHERE name = %s", (new_phone, name))
    else:
        print("Invalid choice")
        return

    conn.commit()
    print("Update complete")
    cur.close()
    conn.close()

def delete_data():
    name = input("Enter name to delete (or leave blank): ")
    phone = input("Enter phone to delete (or leave blank): ")
    conn = psycopg2.connect(dbname='PhoneBook', user='postgres', password='1234', host='localhost', port='5432')
    cur = conn.cursor()

    if name and phone:
        cur.execute("DELETE FROM phonebook WHERE name = %s OR phone = %s", (name, phone))
    elif name:
        cur.execute("DELETE FROM phonebook WHERE name = %s", (name,))
    elif phone:
        cur.execute("DELETE FROM phonebook WHERE phone = %s", (phone,))
    else:
        print("Nothing to delete")
        return

    conn.commit()
    print(f"Deleted {cur.rowcount} record(s)")
    cur.close()
    conn.close()

def main_menu():
    while True:
        print("\nPhonebook Menu:")
        print("1. Create table")
        print("2. Insert data from console")
        print("3. Insert data from CSV")
        print("4. Update data")
        print("5. Query data")
        print("6. Delete data")
        print("7. Exit")
        choice = input("Choose option (1-7): ")

        if choice == '1':
            create_table()
        elif choice == '2':
            insert_from_console()
        elif choice == '3':
            filename = input("Enter CSV filename: ")
            insert_from_csv(filename)
        elif choice == '4':
            update_data()
        elif choice == '5':
            query_data()
        elif choice == '6':
            delete_data()
        elif choice == '7':
            print("Exiting...")
            break
        else:
            print("Invalid choice. Try again.")

if __name__ == '__main__':
    main_menu()


Phonebook Menu:
1. Create table
2. Insert data from console
3. Insert data from CSV
4. Update data
5. Query data
6. Delete data
7. Exit


Choose option (1-7):  2
Your name:  test
Your phone number:  83737373


New user added

Phonebook Menu:
1. Create table
2. Insert data from console
3. Insert data from CSV
4. Update data
5. Query data
6. Delete data
7. Exit


Choose option (1-7):  5


1. Search by pattern
2. Show all records
3. Show paginated records


Choose option (1-3):  2


ID: 2, Name: Temirlan, Phone: 87785926131
ID: 1, Name: Mark, Phone: 87475057993
ID: 3, Name: Ali WhatsApp, Phone: 87086543612
ID: 4, Name: Mama Tel2, Phone: 87074642507
ID: 5, Name: test, Phone: 83737373

Phonebook Menu:
1. Create table
2. Insert data from console
3. Insert data from CSV
4. Update data
5. Query data
6. Delete data
7. Exit


Choose option (1-7):  7


Exiting...
